In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_kan
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/'

    train_data = read_task(dataset_location , split = 'kan_train_negative_augmented')
    dev_data = read_task(dataset_location , split = 'kan_sentiment_dev')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_kan
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['xlm-roberta-base']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Kannada' + 'Augmented' +str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 1.159081220626831
DEV ACC: 0.5267643540669856
DEV Precision: 0.322169908304161
DEV Recall: 0.4202151847706661
DEV F1Score: 0.3539354568637499
BEST ACCURACY -->  DEV: 0.52676
BEST PRECISION -->  DEV: 0.32217
BEST RECALL -->  DEV: 0.42022
BEST F1SCORE -->  DEV: 0.35394
TIME PER EPOCH: 5.5466528058052065

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHE

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 1.2340129613876343
DEV ACC: 0.46710526315789475
DEV Precision: 0.290851660223318
DEV Recall: 0.3922277164621662
DEV F1Score: 0.31562064852025074
BEST ACCURACY -->  DEV: 0.46711
BEST PRECISION -->  DEV: 0.29085
BEST RECALL -->  DEV: 0.39223
BEST F1SCORE -->  DEV: 0.31562
TIME PER EPOCH: 5.557256988684336

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSI

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 1.2743011713027954
DEV ACC: 0.5484449760765551
DEV Precision: 0.3971184326020411
DEV Recall: 0.379824625213733
DEV F1Score: 0.36039000370545327
BEST ACCURACY -->  DEV: 0.54844
BEST PRECISION -->  DEV: 0.39712
BEST RECALL -->  DEV: 0.37982
BEST F1SCORE -->  DEV: 0.36039
TIME PER EPOCH: 5.569608088334402

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIH

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 1.4172512292861938
DEV ACC: 0.46994617224880386
DEV Precision: 0.3067247092315697
DEV Recall: 0.3733419484555849
DEV F1Score: 0.3145437880505748
BEST ACCURACY -->  DEV: 0.46995
BEST PRECISION -->  DEV: 0.30672
BEST RECALL -->  DEV: 0.37334
BEST F1SCORE -->  DEV: 0.31454
TIME PER EPOCH: 5.552435990174612

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSI

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 0.9961236715316772
DEV ACC: 0.5624252392344498
DEV Precision: 0.46650281833436913
DEV Recall: 0.528620922850799
DEV F1Score: 0.4571193989422308
BEST ACCURACY -->  DEV: 0.57895
BEST PRECISION -->  DEV: 0.4665
BEST RECALL -->  DEV: 0.52862
BEST F1SCORE -->  DEV: 0.45712
TIME PER EPOCH: 5.5606165806452434

Training epoch: 3
FINSIHED BATCH: 19 of 246
FINSIHE

In [6]:
print(best_dev_acc)

0.6531848086124402


In [7]:
print(best_epoch)

7


In [8]:
print(best_dev_precision)

0.5395001094536699


In [9]:
print(best_dev_recall)

0.5487712206094559


In [10]:
print(best_dev_f1score)

0.5105609975432119


In [11]:
print(overall_list_dev_acc)

[[0.5267643540669856, 0.6054874401913876, 0.6137858851674641, 0.652811004784689, 0.6366626794258373, 0.6300089712918661, 0.6561004784688995, 0.6143092105263158, 0.6599132775119617, 0.6575209330143541], [0.46710526315789475, 0.5926285885167464, 0.653782894736842, 0.6399521531100478, 0.6433163875598086, 0.6499700956937798, 0.6470544258373206, 0.6480263157894737, 0.6470544258373206, 0.6675388755980861], [0.5484449760765551, 0.6285885167464115, 0.62813995215311, 0.6413726076555024, 0.6067583732057416, 0.6404754784688995, 0.6507924641148325, 0.6475777511961722, 0.6457087320574163, 0.6522876794258373], [0.46994617224880386, 0.5974132775119617, 0.5964413875598086, 0.6276913875598086, 0.6309808612440192, 0.6290370813397129, 0.6555771531100478, 0.6418211722488039, 0.6413726076555024, 0.6627541866028708], [0.5789473684210527, 0.5624252392344498, 0.6039174641148325, 0.6191686602870813, 0.6460077751196173, 0.6531848086124402, 0.6395035885167464, 0.6285885167464115, 0.6219348086124402, 0.6423444976

In [12]:
print(overall_list_dev_precision)

[[0.322169908304161, 0.46780473110519893, 0.49143773060718243, 0.47184558264701804, 0.4889120548315243, 0.46704122271800846, 0.5241574721973442, 0.4907600606464243, 0.542235498721934, 0.5431706593590918], [0.290851660223318, 0.422094067548613, 0.5269785335142075, 0.49671537655670783, 0.4937169532159681, 0.4935412899949911, 0.49284201745185424, 0.5182672619270213, 0.5051557555244579, 0.5473285259749164], [0.3971184326020411, 0.4728374539541198, 0.4945513576007402, 0.49683104943888096, 0.5094776150023477, 0.4819406091718527, 0.47702110004952664, 0.5181262460371665, 0.5180912786353381, 0.5283674996958666], [0.3067247092315697, 0.4749421148115492, 0.47007846168045314, 0.4740814859531437, 0.49378834336458177, 0.47470535728624424, 0.5364133340625037, 0.5072161754614964, 0.5145533182039227, 0.5429350518215561], [0.4171841278893418, 0.46650281833436913, 0.49635813050585775, 0.46344553383024395, 0.5111379841583191, 0.512457269409141, 0.4911605592510814, 0.5395001094536699, 0.5089574685563991, 0

In [13]:
print(overall_list_dev_recall)

[[0.4202151847706661, 0.4935860280598003, 0.5036806746432414, 0.5203239165141078, 0.5271348883236724, 0.4580270300031769, 0.5393774889425687, 0.5230926752985576, 0.5412827895905576, 0.5326409773850315], [0.3922277164621662, 0.4690148108294745, 0.5101832705009597, 0.5327585081053979, 0.5173223538489687, 0.5348780178526167, 0.5312251001492488, 0.5417047068155286, 0.5262261732857004, 0.5625282332906173], [0.379824625213733, 0.49345287993349485, 0.48728176643617815, 0.5124570865307566, 0.5465048610035242, 0.5090352910606921, 0.49643113594351984, 0.5204997688617164, 0.5164589529056176, 0.5284385682647714], [0.3733419484555849, 0.45480412257669145, 0.47107754030416227, 0.4923044003015154, 0.5334113752780195, 0.49287376956002055, 0.5455061432844294, 0.5437298091602553, 0.5489361425858751, 0.5615958788686061], [0.4441238733785525, 0.528620922850799, 0.5383856914577432, 0.5155163371825346, 0.5487712206094559, 0.5141675428661778, 0.5284602656545382, 0.5280821396213318, 0.5301877296797083, 0.5342

In [14]:
print(overall_list_dev_f1score)

[[0.3539354568637499, 0.4511104289190429, 0.47459148240081284, 0.4756758377666907, 0.48366556833676166, 0.42795651283236824, 0.5079351442149934, 0.4815218099181532, 0.517103247336513, 0.5185090487994144], [0.31562064852025074, 0.4186928346658624, 0.4910313679831671, 0.4975629787169845, 0.4829095438531898, 0.48756789083409796, 0.4941904442125635, 0.5091769198182199, 0.49289688423788897, 0.5299022499841155], [0.36039000370545327, 0.4623876302135463, 0.4619595728877879, 0.4850067039231139, 0.49742961813545283, 0.4752726790479398, 0.4649474833554745, 0.4970321141781296, 0.49591500112836095, 0.505891786049935], [0.3145437880505748, 0.43726729898846645, 0.44813922591429606, 0.4577398479832795, 0.4846876358533196, 0.4678756189728867, 0.5150593934992831, 0.4942928240429503, 0.5130564556903331, 0.5299589458138345], [0.40791524566403453, 0.4571193989422308, 0.4815870972063552, 0.47029821374813535, 0.5048756087616076, 0.4903411176873727, 0.490186028248781, 0.5105609975432119, 0.4967306091358402, 

In [15]:
#The best model is 